In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals
import pandas
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
from keras.utils import np_utils

#from sklearn.model_selection import train_test_split
#from sklearn.datasets import make_circles

from keras import backend as K
import matplotlib.pyplot as plt
import re
import hexdump

print("TensorFlow version: {}".format(tf.__version__))

TensorFlow version: 2.7.0


In [2]:
X , Y  = make_circles(n_samples=500, factor=0.5, noise=0.05)
X = 5*X

plt.scatter(X[Y==0,0],X[Y==0,1],c="skyblue")
plt.scatter(X[Y==1,0],X[Y==1,1],c="red")

print(np.shape(X))
print(np.shape(Y))

NameError: name 'make_circles' is not defined

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,Y,train_size=0.8,random_state=23)


X_train_1 = K.cast_to_floatx (X_train)
X_test_1 = K.cast_to_floatx (X_test)
y_train_1 = K.cast_to_floatx (y_train)

In [ ]:
# El modelo
model= tf.keras.Sequential()

model.add(layers.Dense(16, activation='relu', input_dim=2))
model.add(layers.Dense(8, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

# Configulación del entrenamiento
model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

model.summary()


In [ ]:
print("Iniciar el entrenamiento")
historial = model.fit(X_train_1,y_train_1, epochs=100,batch_size=1,verbose=True)
print("Modelo Entrenado")

plt.xlabel("# Epoca")
plt.ylabel("Magnitud de perdida")
plt.plot(historial.history["loss"])

In [ ]:
loss, accuracy = model.evaluate(X_test_1, y_test, verbose=0)
print("Test accuracy : " + str(accuracy))

In [ ]:
def port(model, optimize=False, variable_name='model_data', pretty_print=False):
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    if optimize:
        if isinstance(optimize, bool):
            optimizers = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
        else:
            optimizers = optimize
        converter.optimizations = optimizers
    tflite_model = converter.convert()
    bytes = hexdump.dump(tflite_model).split(' ')
    c_array = ', '.join(['0x%02x' % int(byte, 16) for byte in bytes])
    c = 'unsigned char %s[] = {%s};' % (variable_name, c_array)
    if pretty_print:
        c = c.replace('{', '{\n\t').replace('}', '\n}')
        c = re.sub(r'(0x..?, ){12}', lambda x: '%s\n\t' % x.group(0), c)
    c += '\nconst int %s_len = %d;' % (variable_name, len(bytes))
    preamble = '''
// if having troubles with min/max, uncomment the following
// #undef min
// #undef max
#ifdef __has_attribute
#define HAVE_ATTRIBUTE(x) __has_attribute(x)
#else
#define HAVE_ATTRIBUTE(x) 0
#endif
#if HAVE_ATTRIBUTE(aligned) || (defined(__GNUC__) && !defined(__clang__))
#define DATA_ALIGN_ATTRIBUTE __attribute__((aligned(4)))
#else
#define DATA_ALIGN_ATTRIBUTE
#endif
'''
    return c

In [ ]:
text = port(model,variable_name="mi_data")
text